Let's now prepare the data for all sites:

In [6]:
import pandas as pd
import numpy as np
import xarray as xr
from read_data import read_data



df_merged, df_avg = read_data("PM25_24g")

df_merged = df_merged.to_xarray()

df_merged.to_netcdf('PM25_24g_merged.nc')

Wczytywanie: .\2002\2002_PM25_24g.xlsx
Wczytywanie: .\2003\2003_PM25_24g.xlsx
Wczytywanie: .\2004\2004_PM25_24g.xlsx
Wczytywanie: .\2005\2005_PM25_24g.xlsx
Wczytywanie: .\2006\2006_PM25_24g.xlsx
Wczytywanie: .\2007\2007_PM25_24g.xlsx
Wczytywanie: .\2008\2008_PM25_24g.xlsx
Wczytywanie: .\2009\2009_PM25_24g.xlsx
Wczytywanie: .\2010\2010_PM25_24g.xlsx
Wczytywanie: .\2011\2011_PM25_24g.xlsx
Wczytywanie: .\2012\2012_PM25_24g.xlsx
Wczytywanie: .\2013\2013_PM25_24g.xlsx
Wczytywanie: .\2014\2014_PM25_24g.xlsx
Wczytywanie: .\2015\2015_PM25_24g.xlsx
Wczytywanie: .\2016\2016_PM25_24g.xlsx
Wczytywanie: .\2017\2017_PM25_24g.xlsx
Wczytywanie: .\2018\2018_PM25_24g.xlsx
Wczytywanie: .\2019\2019_PM25_24g.xlsx
Wczytywanie: .\2020\2020_PM25_24g.xlsx
Wczytywanie: .\2021\2021_PM25_24g.xlsx
Wczytywanie: .\2022\2022_PM25_24g.xlsx
Wczytywanie: .\2023\2023_PM25_24g.xlsx
            MpKrakBulwar  MpKrakowWIOSPrad6115  LdLodzLegion  \
Data                                                           
2002-01-01    

d:\MOJE\ERA5+MLwalbrzych\read_data.py:105: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df_merged = df_renamed.groupby(axis=1, level=0).mean(numeric_only=True)


Now we use only data from some time fragment, cause we don't want too much data because of computational time.

Here i used the criteria like in the

> — Vovk, T., Kryza, M., & Werner, M., *Using Random Forest to improve EMEP4PL model estimates of daily PM₂.₅ in Poland* (2024) :contentReference[oaicite:0]{index=0}

In [7]:
import pandas as pd
import xarray as xr

# === PARAMETERS ===
START_YEAR = 2016
END_YEAR = 2019
MIN_YEARS = 3
MIN_MONTHS_PER_YEAR = 12
MIN_DAYS_PER_MONTH = 14

# === 1. Load data and restrict to years 2016–2019 ===
df = xr.open_dataset('PM25_24g_merged.nc').to_dataframe()
df.index.name = 'date'
df = df.sort_index()

# Ensure index is datetime
df.index = pd.to_datetime(df.index)

# Only keep years 2016–2019
df = df[(df.index.year >= START_YEAR) & (df.index.year <= END_YEAR)]

# === 2. Function: check station completeness ===
def is_station_complete(series: pd.Series) -> bool:
    # Remove NaN values
    s = series.dropna()
    if s.empty:
        return False
    
    # Group by: year → month → number of measured days
    counts = s.groupby([s.index.year, s.index.month]).size()
    
    # Count years that meet the monthly criteria
    valid_years = 0
    for year in range(START_YEAR, END_YEAR + 1):
        # Days per month for this year
        days_per_month = counts.loc[year] if year in counts.index.get_level_values(0) else pd.Series(dtype=int)
        # Number of months with >= MIN_DAYS_PER_MONTH
        good_months = (days_per_month >= MIN_DAYS_PER_MONTH).sum()
        if good_months >= MIN_MONTHS_PER_YEAR:
            valid_years += 1
    
    # Check yearly completeness criterion
    return valid_years >= MIN_YEARS

# === 3. Filter stations ===
cleaned = {}
for station in df.columns:
    if is_station_complete(df[station]):
        cleaned[station] = df[station]

# === 4. Merge and save ===
if cleaned:
    df_clean = pd.DataFrame(cleaned)
    df_clean = df_clean.dropna(how='all')
    print(f"Kept {len(df_clean.columns)} stations meeting the criteria.")
    df_clean.to_csv('stations_complete_2016_2019.csv')
else:
    print("No stations met the criteria.")


Kept 61 stations meeting the criteria.


We also prepare 1 hour data for later:

In [4]:
import pandas as pd
import numpy as np
import xarray as xr
from read_data_hour import read_data

df_merged, df_avg = read_data("PM25_1g")

df_merged = df_merged.to_xarray()

df_merged.to_netcdf('PM25_1g_merged.nc')

Wczytywanie: .\2006\2006_PM25_1g.xlsx
Wczytywanie: .\2007\2007_PM25_1g.xlsx
Wczytywanie: .\2008\2008_PM25_1g.xlsx
Wczytywanie: .\2009\2009_PM25_1g.xlsx
Wczytywanie: .\2010\2010_PM25_1g.xlsx
Wczytywanie: .\2011\2011_PM25_1g.xlsx
Wczytywanie: .\2012\2012_PM25_1g.xlsx
Wczytywanie: .\2013\2013_PM25_1g.xlsx
Wczytywanie: .\2014\2014_PM25_1g.xlsx


c:\Users\szymo\anaconda3\envs\geo\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


Wczytywanie: .\2015\2015_PM25_1g.xlsx
Wczytywanie: .\2016\2016_PM25_1g.xlsx
Wczytywanie: .\2017\2017_PM25_1g.xlsx
Wczytywanie: .\2018\2018_PM25_1g.xlsx
Wczytywanie: .\2019\2019_PM25_1g.xlsx
Wczytywanie: .\2020\2020_PM25_1g.xlsx
Wczytywanie: .\2021\2021_PM25_1g.xlsx
Wczytywanie: .\2022\2022_PM25_1g.xlsx
Wczytywanie: .\2023\2023_PM25_1g.xlsx
                     ZpSzczPilsud  KpBydWarszaw  MpKrakBulwar  \
Data                                                            
2006-01-03 01:00:00           NaN           NaN           NaN   
2006-01-03 02:00:00           NaN           NaN           NaN   
2006-01-03 03:00:00           NaN           NaN           NaN   
2006-01-03 04:00:00           NaN           NaN           NaN   
2006-01-03 05:00:00           NaN           NaN           NaN   
...                           ...           ...           ...   
2023-12-31 20:00:00           NaN           NaN           NaN   
2023-12-31 21:00:00           NaN           NaN           NaN   
2023-12-

d:\MOJE\ERA5+MLwalbrzych\read_data_hour.py:105: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df_merged = df_renamed.groupby(axis=1, level=0).mean(numeric_only=True)


                     DsDusznikMOB  DsDzialoszyn  DsJaworMOB  DsJedlinaZdrMOB  \
Data                                                                           
2006-01-03 01:00:00           NaN           NaN         NaN              NaN   
2006-01-03 02:00:00           NaN           NaN         NaN              NaN   
2006-01-03 03:00:00           NaN           NaN         NaN              NaN   
2006-01-03 04:00:00           NaN           NaN         NaN              NaN   
2006-01-03 05:00:00           NaN           NaN         NaN              NaN   
...                           ...           ...         ...              ...   
2023-12-31 20:00:00           NaN           NaN         NaN              NaN   
2023-12-31 21:00:00           NaN           NaN         NaN              NaN   
2023-12-31 22:00:00           NaN           NaN         NaN              NaN   
2023-12-31 23:00:00           NaN           NaN         NaN              NaN   
2024-01-01 00:00:00           NaN       

In [5]:
import pandas as pd
import xarray as xr

# === PARAMETERS ===
START_YEAR = 2016
END_YEAR = 2019
MIN_YEARS = 3
MIN_MONTHS_PER_YEAR = 12
MIN_DAYS_PER_MONTH = 14

# === 1. Load data and restrict to years 2016–2019 ===
df = xr.open_dataset('PM25_1g_merged.nc').to_dataframe()
df.index.name = 'date'
df = df.sort_index()

# Ensure index is datetime
df.index = pd.to_datetime(df.index)

# Only keep years 2016–2019
df = df[(df.index.year >= START_YEAR) & (df.index.year <= END_YEAR)]

# === 2. Function: check station completeness ===
def is_station_complete(series: pd.Series) -> bool:
    # Remove NaN values
    s = series.dropna()
    if s.empty:
        return False
    
    # Group by: year → month → number of measured days
    counts = s.groupby([s.index.year, s.index.month]).size()
    
    # Count years that meet the monthly criteria
    valid_years = 0
    for year in range(START_YEAR, END_YEAR + 1):
        # Days per month for this year
        days_per_month = counts.loc[year] if year in counts.index.get_level_values(0) else pd.Series(dtype=int)
        # Number of months with >= MIN_DAYS_PER_MONTH
        good_months = (days_per_month >= MIN_DAYS_PER_MONTH).sum()
        if good_months >= MIN_MONTHS_PER_YEAR:
            valid_years += 1
    
    # Check yearly completeness criterion
    return valid_years >= MIN_YEARS

# === 3. Filter stations ===
cleaned = {}
for station in df.columns:
    if is_station_complete(df[station]):
        cleaned[station] = df[station]

# === 4. Merge and save ===
if cleaned:
    df_clean = pd.DataFrame(cleaned)
    df_clean = df_clean.dropna(how='all')
    print(f"Kept {len(df_clean.columns)} stations meeting the criteria.")
    df_clean.to_csv('stations_complete_2016_2019_1h.csv')
else:
    print("No stations met the criteria.")

Kept 36 stations meeting the criteria.
